# 🚀 NovaGen AI - Backend Server (Repo Edition)
Este notebook automatizado despliega el backend de NovaGen AI en Google Colab T4.

- 🧠 **Gestión de Modelos**: Descarga Juggernaut-XL v9 y LoRAs personalizados.
- 📥 **Clonado Automático**: Obtiene la última versión de `Juanpalojime/NovaGenIA`.
- ⚡ **GPU T4**: Verificación y optimización de entorno.
- 🌐 **Acceso Remoto**: Túnel Ngrok seguro.
- 🧠 **Gestión de Modelos**: Descarga SDXL 1.0 y LoRAs personalizados.
- 🔌 **API FastAPI**: Endpoints para generación de imágenes y entrenamiento.

In [ ]:
# @title 1. Inicialización del Entorno (Clone & Install)
import os
import subprocess
import sys

print("🚀 Iniciando secuencia de arranque...")

# 1. Verificar GPU
import torch
if torch.cuda.is_available():
    print(f"✅ GPU Detectada: {torch.cuda.get_device_name(0)}")
    # Force reinstall for xformers compatibility if needed
else:
    print("⚠️ ADVERTENCIA: GPU no detectada. Ve a 'Entorno de ejecución' > 'Cambiar tipo' > 'T4 GPU'.")

# 2. Clonar/Actualizar Repositorio
REPO_URL = "https://github.com/Juanpalojime/NovaGenIA.git"
REPO_NAME = "NovaGenIA"

if os.path.exists(REPO_NAME):
    print("🔄 Actualizando repositorio...")
    %cd {REPO_NAME}
    !git pull
    %cd ..
else:
    print("⬇ Clonando repositorio...")
    !git clone {REPO_URL}

# 3. Instalar Dependencias
print("⏳ Instalando dependencias de Python (aprox. 2 min)...")
!pip install -r {REPO_NAME}/requirements.txt -q
!pip install pyngrok nest_asyncio huggingface_hub websockets -q

print("✅ Entorno preparado.")

In [ ]:
# @title 1.5 Mount Google Drive (Optional)
from google.colab import drive
import os

print("📂 Mounting Google Drive...")
try:
    drive.mount('/content/drive')
    print("✅ Drive mounted at /content/drive")
    
    # Create outputs folder if not exists
    drive_output_path = "/content/drive/MyDrive/NovaGen_Outputs"
    os.makedirs(drive_output_path, exist_ok=True)
    print(f"✅ Output folder ready: {drive_output_path}")
except Exception as e:
    print(f"⚠️ Drive mount skipped or failed: {e}")

In [ ]:
# @title 1.8 Build Frontend (Install Node.js & Build React App)
import os

print("📦 Installing Node.js 18...")
!curl -fsSL https://deb.nodesource.com/setup_18.x | sudo -E bash - > /dev/null
!sudo apt-get install -y nodejs > /dev/null

print("📂 Building Frontend App...")
frontend_dir = f"{REPO_NAME}/app"
if os.path.exists(frontend_dir):
    %cd {frontend_dir}
    print("   installing npm packages...")
    !npm install > /dev/null 2>&1
    print("   building production bundle...")
    !npm run build
    %cd ../..
    print("✅ Frontend Build Complete!")
else:
    print(f"❌ Frontend directory not found at {frontend_dir}")

In [ ]:
# @title 2. Configuración de Credenciales (Ngrok, HuggingFace, CivitAI)
from pyngrok import ngrok
import os

# @markdown ### Ngrok Setup
# @markdown Token para túnel público (https://dashboard.ngrok.com):
NGROK_AUTH_TOKEN = "" # @param {type:"string"}

# @markdown ### Model Hubs Auth
# @markdown Token de HuggingFace (Lectura) para descargar SDXL:
HF_TOKEN = "" # @param {type:"string"}
# @markdown API Key de CivitAI (Opcional para LoRAs restringidos):
CIVITAI_API_KEY = "" # @param {type:"string"}

# Configurar Ngrok
if NGROK_AUTH_TOKEN:
    # Limpieza automática por si se pega el comando completo
    clean_token = NGROK_AUTH_TOKEN.replace("ngrok config add-authtoken ", "").strip()
    ngrok.set_auth_token(clean_token)
    print(f"✅ Ngrok configurado (Token: {clean_token[:4]}...{clean_token[-4:]})")
else:
    print("⚠️ Ngrok Token no proporcionado. El túnel podría fallar.")

# Guardar variables para uso en el sistema
os.environ["HF_TOKEN"] = HF_TOKEN
os.environ["CIVITAI_API_KEY"] = CIVITAI_API_KEY

if HF_TOKEN:
    !huggingface-cli login --token {HF_TOKEN}
    print("✅ HuggingFace Login Exitoso.")

In [ ]:
# @title 3. Descarga de Modelos (Juggernaut-XL v9 & LoRAs)
import os

MODELS_DIR = f"{REPO_NAME}/models/checkpoints"
LORAS_DIR = f"{REPO_NAME}/models/loras"
os.makedirs(MODELS_DIR, exist_ok=True)
os.makedirs(LORAS_DIR, exist_ok=True)

print("📥 Comenzando descarga de modelos...")

# 1. Juggernaut-XL v9 RunDiffusion Photo v2 desde CivitAI
print("   - Descargando Juggernaut-XL v9 RunDiffusion Photo v2...")
MODEL_FILENAME = "Juggernaut-XL_v9_RunDiffusionPhoto_v2.safetensors"
MODEL_PATH = os.path.join(MODELS_DIR, MODEL_FILENAME)

if not os.path.exists(MODEL_PATH):
    # URL de CivitAI para Juggernaut-XL v9
    # Nota: Esta es la URL directa del modelo. Si requiere API key, agrégala como parámetro
    MODEL_URL = "https://civitai.com/api/download/models/456194"
    
    if os.environ.get("CIVITAI_API_KEY"):
        MODEL_URL = f"{MODEL_URL}?token={os.environ['CIVITAI_API_KEY']}"
    
    print(f"     ⬇ Descargando {MODEL_FILENAME} (~6.5 GB, puede tardar 5-10 min)...")
    !wget -O {MODEL_PATH} "{MODEL_URL}" --progress=bar:force:noscroll
    print(f"     ✅ {MODEL_FILENAME} descargado exitosamente.")
else:
    print(f"     ✅ {MODEL_FILENAME} ya existe.")

# 2. Descargar LoRAs de CivitAI
loras_to_download = [
    # { "name": "pixel_art_xl.safetensors", "url": "https://civitai.com/api/download/models/123456" },
    # Añade tus URLs aquí
]

print(f"   - Descargando {len(loras_to_download)} LoRAs adicionales...")
for lora in loras_to_download:
    name = lora['name']
    url = lora['url']
    path = os.path.join(LORAS_DIR, name)
    
    if not os.path.exists(path):
        if os.environ.get("CIVITAI_API_KEY"):
            url = f"{url}?token={os.environ['CIVITAI_API_KEY']}"
        
        print(f"     ⬇ Descargando {name}...")
        !wget -O {path} "{url}"
    else:
        print(f"     ✅ {name} ya existe.")

# 3. Descargar Modelo FaceSwap (InsightFace)
print("   - Descargando modelo FaceSwap (inswapper_128.onnx)...")
FACESWAP_DIR = f"{REPO_NAME}/models/insightface"
os.makedirs(FACESWAP_DIR, exist_ok=True)
SWAP_MODEL_URL = "https://huggingface.co/ezioruan/inswapper_128.onnx/resolve/main/inswapper_128.onnx"
SWAP_MODEL_PATH = os.path.join(FACESWAP_DIR, "inswapper_128.onnx")

if not os.path.exists(SWAP_MODEL_PATH):
    print(f"     ⬇ Descargando inswapper_128.onnx...")
    !wget -O {SWAP_MODEL_PATH} "{SWAP_MODEL_URL}" --progress=bar:force:noscroll
    print(f"     ✅ inswapper_128.onnx descargado.")
else:
    print(f"     ✅ inswapper_128.onnx ya existe.")

print("🏁 Descargas completadas.")

In [ ]:
# @title 4. INICIAR BACKEND (FastAPI + Uvicorn)
import nest_asyncio
import time
from pyngrok import ngrok
from pyngrok.exception import PyngrokNgrokError
import sys

nest_asyncio.apply()

print("🔄 Limpiando procesos anteriores...")
!pkill -f uvicorn
ngrok.kill()
time.sleep(3) # Esperar a que se cierren completamente

# Iniciar Túnel
PORT = 7860
try:
    print("🔌 Conectando Ngrok...")
    tunnel = ngrok.connect(PORT)
    public_url = tunnel.public_url

    print("="*50)
    print(f"🔗 PUBLIC API URL: \033[96m{public_url}\033[0m")
    print("="*50)
    print("👉 Copia esta URL y pégala en NovaGen > Settings > Connections")
    print("👉 El servidor iniciará a continuación (Logs abajo)...\n")
    print("⚠️ NOTA: Como el frontend está servido en / (root), tu URL pública muestra la App directamente.")

    # Cambiar al directorio del repo para ejecución
    %cd {REPO_NAME}
    
    # Patch para correr tests si se desea (opcional)
    # !pytest tests/

    # Iniciar Servidor (Bloqueante)
    !python server.py
except PyngrokNgrokError as e:
    print("\n❌ ERROR DE NGROK:")
    if "ERR_NGROK_108" in str(e):
        print("⚠️ Tu cuenta gratuita de Ngrok tiene una sesión ya activa.")
        print("👉 Ve a https://dashboard.ngrok.com/agents y elimina la sesión existente.")
    else:
        print(f"Detalles: {e}")